<a href="https://colab.research.google.com/github/alexarroyo99/subs_gen/blob/main/Subs_generator_by_Alex_Arroyo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title 🔧 Configurarcion inicial

!pip install git+https://github.com/openai/whisper.git &> /dev/null
!pip install jiwer &> /dev/null
!pip install moviepy --upgrade &> /dev/null
!apt-get update &> /dev/null
!apt-get -y install ffmpeg imagemagick &> /dev/null
!export FFMPEG_BINARY='/usr/bin/ffmpeg' &> /dev/null
!export IMAGEMAGICK_BINARY='/usr/bin/convert' &> /dev/null
!sed -i '70d' ~/../etc/ImageMagick-6/policy.xml &> /dev/null
!sed -i '71d' ~/../etc/ImageMagick-6/policy.xml &> /dev/null
!sed -i '40d' ~/../etc/ImageMagick-6/policy.xml &> /dev/null

import whisper
import cv2
import pandas as pd
from moviepy.editor import *
from moviepy.video.tools.subtitles import SubtitlesClip

def gen_subs(uploaded_video):
  vid = VideoFileClip(uploaded_video)
  vid.audio.write_audiofile("audio.mp3")

  model = whisper.load_model("medium")
  result = model.transcribe("audio.mp3")

  dict1 = {'start':[], 'end':[], 'text':[]}
  for i in result['segments']:
      dict1['start'].append(int(i['start']))
      dict1['end'].append(int(i['end']))
      dict1['text'].append(i['text'])
  df = pd.DataFrame.from_dict(dict1)
  df.to_csv("subs.csv")
  vidcap = cv2.VideoCapture(uploaded_video)
  # success,image = vidcap.read()
  # height = image.shape[0]
  # width =image.shape[1]

  generator = lambda txt: TextClip(txt, color='white', fontsize=20, font='Georgia-Regular',stroke_width=3, method='caption', align='south', size=vid.size)
  subs = tuple(zip(tuple(zip(df['start'].values, df['end'].values)), df['text'].values))
  subtitles = SubtitlesClip(subs, generator)

  final = CompositeVideoClip([vid, subtitles.set_pos(('center','bottom'))])
  final.write_videofile('output.mp4', fps=vid.fps, remove_temp=True, codec="libx264", audio_codec="aac")

In [ ]:
#@title 🎥 Cargar video
from google.colab import files

uploaded = files.upload()

video = list(uploaded.keys())[0]

In [ ]:
#@title 🔤 Generar subtitulos
from IPython.display import HTML
from base64 import b64encode

gen_subs(video)

mp4 = open('output.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)